In [44]:
from collections import defaultdict
from collections import namedtuple
import time
import random
import json
import string
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
import h5py
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
from heapq import nlargest
import dill
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import requests
from io import BytesIO
from PIL import Image
import PIL
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image, ImageDraw, ImageFont
import PIL
import os
import matplotlib.pyplot as plt
torch.manual_seed(1)
random.seed(1)
torch.__version__

'0.3.0.post4'

In [45]:
CUDA = torch.cuda.is_available()
print(CUDA)

True


In [46]:
IMG_FEATURES = "./images/IR_image_features.h5"
IMG_ID = "./images/IR_img_features2id.json"

TRAIN_HARD = "./Data/Hard/IR_train_hard.json"
TRAIN_EASY = "./Data/Easy/IR_train_easy.json"
TEST_HARD = "./Data/Hard/IR_test_hard.json"
TEST_EASY = "./Data/Easy/IR_test_easy.json"
VAL_HARD = "./Data/Hard/IR_val_hard.json"
VAL_EASY = "./Data/Easy/IR_val_easy.json"
IMGID2IMGINFO = "./Data/imgid2imginfo.json"


BATCH_SIZE = 100
LEARNING_RATE = 1e-3
EPOCHS = 100
CAPTION_DICT = {}
CURRENT_MODEL_NAME = "LSTM_HARD"

In [47]:
def write(x, name):
    import pickle
    file_name = './results/' + CURRENT_MODEL_NAME + '/' + name + '.npy'
    np.save(file_name, np.array(x))

In [48]:
#GET features from images
with open("./images/IR_img_features2id.json", 'r') as f:
    visual_feat_mapping = json.load(f)['IR_imgid2id']
f.close()

img_features = np.asarray( h5py.File("./images/IR_image_features.h5", 'r')['img_features'])

def get_feature_from_id(img_id):
    h5_id = visual_feat_mapping[str(img_id)]
    return img_features[h5_id]

In [49]:
""" Helper function to show an image, from a image id """
#source: https://stackoverflow.com/questions/34255938/is-there-a-way-to-specify-the-width-of-a-rectangle-in-pil
def draw_rectangle(draw, coordinates, color, width=100):
    for i in range(width):
        rect_start = (coordinates[0][0] - i, coordinates[0][1] - i)
        rect_end = (coordinates[1][0] + i, coordinates[1][1] + i)
        draw.rectangle((rect_start, rect_end), outline = color)

def show_img_from_id(img_ids, target_id = -1):
    
    try:
        with open(IMGID2IMGINFO, 'r') as f:
            imgid2info = json.load(f)

        response = requests.get(imgid2info[str(img_ids[0])]['coco_url'])
        img = Image.open(BytesIO(response.content))
        imgs = [img, img]
        width, height = img.size
        draw = ImageDraw.Draw(img)
        line_width = 30
        draw.rectangle(((0, 0), (width*2, height*2)), fill="white")
        for img_id in img_ids:
            response = requests.get(imgid2info[str(img_id)]['coco_url'])
            img = Image.open(BytesIO(response.content))
            if(img_id == target_id):
                width, height = img.size
                draw = ImageDraw.Draw(img)
                line_width = 30
                draw_rectangle(draw, coordinates=((line_width, line_width), (width - line_width, height - line_width)), color="green", width=line_width)
            imgs.append(img)

        f.close()

        min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[-1][1]
        imgs_comb = np.hstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )

        # save that beautiful picture
        imgs_comb = PIL.Image.fromarray( imgs_comb)
        width, height = imgs_comb.size
        draw = ImageDraw.Draw(imgs_comb)
        if(target_id != -1):
            # fnt = ImageFont.truetype("/Library/Fonts/Comic Sans MS.ttf", 30) 
            fnt = ImageFont.truetype('Pillow/Tests/fonts/FreeMono.ttf', 30)
            draw.multiline_text((10, 10), CAPTION_DICT[target_id].replace("?", "?") ,(0,0,0), font=fnt)

        files = "["+str(target_id)+"]_"
        files += '_'.join(map(str, img_ids))
        if(img_ids[0] == target_id):
            file_name = './results/' + CURRENT_MODEL_NAME + '/correct/top1/'
        elif(img_ids[1] == target_id or img_ids[2] == target_id or img_ids[3] == target_id or img_ids[4] == target_id):
            file_name = './results/' + CURRENT_MODEL_NAME + '/correct/top5/'  
        else:
            file_name = './results/' + CURRENT_MODEL_NAME + '/wrong/' 
        imgs_comb.save( file_name + files + '.jpg' )
    except:
        import traceback
        print( traceback.format_exc())
        pass
    
show_img_from_id([378466, 378466, 378466, 378466], 378466)

Traceback (most recent call last):
  File "<ipython-input-49-b0ba592f3fd4>", line 44, in show_img_from_id
    draw.multiline_text((10, 10), CAPTION_DICT[target_id].replace("?", "?") ,(0,0,0), font=fnt)
KeyError: 378466



In [50]:

def create_learning_curves(top1_val_list, top5_val_list, top1_train_list, top5_train_list):
    fig = plt.figure()
    ax = plt.subplot(111)

    
    ax.plot(top5_val_list, label='top5 validation', c='darkred')
    ax.plot(top5_train_list, label='top5 train', c='royalblue')
    
    ax.plot(top1_val_list, linestyle=':', label='top1 validation', c='darkred')
    ax.plot(top1_train_list, linestyle=':', label='top1 train', c='royalblue')
    
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.75, box.height])

    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.title('Learning curve of LSTM Easy')
    plt.ylabel('Accuracy')
    plt.xlabel('Number of iterations')
    plt.savefig('./results/' + CURRENT_MODEL_NAME + '/accuracy1.png')
    plt.clf()
    plt.cla()
    plt.close()


In [51]:
"""
Preprocess a sentence
"""
def preprocess(sentence, stemmer, stop):
    low_sent = sentence.lower()
    # Possibility to tokenize entire dataset and put in counter to filter out
    # infrequent/frequent words
    tok_sent = word_tokenize(low_sent)
    stop_stem_sent = [stemmer.stem(i) for i in tok_sent if i not in stop]
    return stop_stem_sent

In [52]:
"""
Convert Samples
"""

def get_dialog_caption_targets_from_sample(sample):
    dialog = ''
    caption = sample['caption']
    targets = []
    targetidx = sample['target']
   
    dialog_text = ''
    for d in sample['dialog']:
        dialog += ' ' + d[0]
        dialog_text += ' ' + d[0] + '\n'

    for img in sample['img_list']:
        targets.append(img)
    
    CAPTION_DICT[targets[targetidx]] = dialog_text + "  \n " + caption
    return dialog, caption, targets, targetidx

Sample = namedtuple("Sample", ["words", "images", "target"])

"""
 For every Sample we retrieve the sentences and the img_ids, and the correct target_id. 
"""
def read_dataset(filename, stemmer, stopwords):
    with open(filename, "r") as f:
        dataset = json.load(f)
    f.close()
    for idx, sample in enumerate(dataset):
        if(idx % 10000 == 0):
            print(idx)
        dialog, caption, targets, targetidx = get_dialog_caption_targets_from_sample(dataset[
                                                                                     str(sample)])
        sentences = preprocess(dialog + ' ' + caption, stemmer, stopwords)
        yield Sample(words=[w2i[x] for x in sentences], images=targets, target=targetidx)



In [53]:
w2i = defaultdict(lambda: len(w2i))
UNK = w2i["<UNK>"]
PAD = w2i["<PAD>"]

stemmer = SnowballStemmer("english")
stop = stopwords.words('english') + list(string.punctuation)


In [54]:
#read the datasets and use w2i (only do this once)
train = list(read_dataset(TRAIN_HARD, stemmer, stop))
w2i = defaultdict(lambda: UNK, w2i)
val = list(read_dataset(VAL_HARD, stemmer, stop))
test = list(read_dataset(TEST_HARD, stemmer, stop))

0
10000
20000
30000
0
0


In [55]:
nwords = len(w2i)

In [56]:
"""
CLASS LSTM 
"""
class LSTMC3(nn.Module):
    def __init__(self, vocab_size, embedding_size, img_feat_dim, hidden_dim_mlp, hidden_dim_lstm, num_layers_lstm,  output_dim, batch_size):
        super(LSTMC3, self).__init__()
        


class LSTMC2(nn.Module):

    def __init__(self, vocab_size, embedding_size, img_feat_dim, hidden_dim_mlp, hidden_dim_lstm, num_layers_lstm,  output_dim, batch_size):
        super(LSTMC2, self).__init__()
        self.hidden_dim = hidden_dim_lstm
        self.batch_size = batch_size
        self.word_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx = PAD)
        #
        self.lstm = nn.LSTM(embedding_size, hidden_dim_lstm,batch_first=True)
        self.linear1 = nn.Linear(hidden_dim_lstm+img_feat_dim, hidden_dim_mlp)
        self.linear2 = nn.Linear(hidden_dim_mlp, output_dim)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        if CUDA:
            return Variable(torch.cuda.FloatTensor(1, self.batch_size, self.hidden_dim).fill_(0)),  Variable(torch.cuda.FloatTensor(1, self.batch_size, self.hidden_dim).fill_(0))
        else:
            return (Variable(torch.zeros(1, self.batch_size, self.hidden_dim)), Variable(torch.zeros(1, self.batch_size, self.hidden_dim)))

    def forward(self, sentence, image_feat, lengths):
        embeds = self.word_embeddings(sentence)

        lstm_out, self.hidden = self.lstm(embeds, self.hidden)

        h = torch.gather(lstm_out,1,lengths.view(-1,1,1).expand(len(lengths),1,self.hidden_dim)-1).repeat(1,10,1)
        #Size lstm_out = 99 x 75 x 300

        #100x10x300
        lin1 = torch.cat((h,image_feat),2)
        lin1 = self.linear1(lin1)
        lin1 = F.relu(lin1)
        lin2 = self.linear2(lin1)
        return lin2

In [57]:
# INIT MODEL AND INIT OPTIMIZER
print("Batch_size: ", BATCH_SIZE, "LEARNING_RATE: ",LEARNING_RATE)
print()

model = LSTMC2(nwords, 300, 2048, 45, 100, 1, 1, BATCH_SIZE)
if CUDA:
    model.cuda()
print(model)

#@TODO we can use a adaptive learnrate for adam
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

Batch_size:  100 LEARNING_RATE:  0.001

LSTMC2(
  (word_embeddings): Embedding(15456, 300, padding_idx=1)
  (lstm): LSTM(300, 100, batch_first=True)
  (linear1): Linear(in_features=2148, out_features=45)
  (linear2): Linear(in_features=45, out_features=1)
)


In [58]:
"""
HELPER FUNCTIONS
"""

def preprocessbatch(batch):
    """ Add zero-padding to a batch. """
    seqs = [sample.words for sample in batch]
    
    lengths = [len(sample.words) for sample in batch]
    
    max_length = max(map(len, seqs))
    seqs = [seq + [PAD] * (max_length - len(seq)) for seq in seqs]

    ims = np.array([[get_feature_from_id(img_id) for img_id in sample.images] for sample in batch])

    idxs = [sample.target for sample in batch]
    
    image_ids = [sample.images for sample in batch]

    return seqs, ims, idxs, image_ids, lengths

def minibatch(data, batch_size=BATCH_SIZE):
    for i in range(0, len(data), batch_size):
        yield data[i:i+batch_size]

def getLongTensor(x):
    tensor = torch.cuda.LongTensor(torch.from_numpy(
        x).cuda()) if CUDA else torch.LongTensor(x)
    return Variable(tensor)

def getFloatTensor(x):
    tensor = torch.cuda.FloatTensor(torch.from_numpy(
        x).cuda()) if CUDA else torch.FloatTensor(x)
    return Variable(tensor)


In [59]:
""" 
FUNCTIONS TO SEE HOW GOOD OUR MODEL DID

@TODO: Not fast 
@TODO: top 5 currently not implemented
"""
def evaluate(model, data, show_n_wrong_images  = 0, show_n_good_images=0):
    top1 = 0
    top5 = 0
    
    counter_good = 0
    counter_wrong = 0
    for batch in minibatch(data):
        seqs, image_features, idxs, image_ids, lengths = preprocessbatch(batch)
        scores = model(getLongTensor(np.array([seqs]))[0], getFloatTensor(image_features), getLongTensor(np.array(lengths)))
        targets = getLongTensor(np.array([idxs]))
        _, predictions = torch.max(scores[:, :,0].data, 1)
        
        
        _, top5_predictions = torch.topk(scores[:, :,0].data, 5, largest=True, sorted=True)
        _, top10_predictions = torch.topk(scores[:, :,0].data, 10, largest=True, sorted=True)

        for i in range(len(top5_predictions)):
            if((targets[0][i].data.cpu().numpy() if CUDA else targets[0][i].data.numpy()) in (top5_predictions[i].cpu().numpy() if CUDA else top5_predictions[i].numpy())):
                idx, = np.where((targets[0][i].data.cpu().numpy() if CUDA else targets[0][i].data.numpy()) == (top5_predictions[i].cpu().numpy() if CUDA else top5_predictions[i].numpy()))
                if idx == 0:
                    top1 += 1
                
                top5 += 1
                
                #Show the images Good examples
                if counter_good < show_n_good_images:
                    image_list = list(np.array(image_ids[i])[(top10_predictions[i].cpu().numpy() if CUDA else top10_predictions[i].numpy())])
                    show_img_from_id(image_list, target_id = int(np.array(image_ids[i])[idxs[i]]))
                    counter_good += 1
            else: 
                #Show the images Bad examples
                if counter_wrong < show_n_wrong_images:
                    image_list = list(np.array(image_ids[i])[(top10_predictions[i].cpu().numpy() if CUDA else top10_predictions[i].numpy())])
                    show_img_from_id(image_list, target_id = int(np.array(image_ids[i])[idxs[i]]))
                    counter_wrong += 1
           
    return top1/len(data), top5/len(data)


In [60]:
"""
    RUNNING THE MODEL!!!!!!!!!!
"""
top1_val_list, top5_val_list, top1_train_list, top5_train_list, train_loss_list = [],[],[],[],[]
try:
    for ITER in range(EPOCHS):
        # Init variable
        random.shuffle(train)
        train_loss = 0.0
        start = time.time()
        updates = 0
        
        for batch in minibatch(train):
            updates += 1
            # pad data with zeros
            seqs, image_features, idxs, _, lengths = preprocessbatch(batch)
            
            #reset hidden layer.1
            #@todo not certain if we need this
            model.hidden = model.init_hidden() 
            
            
            # forward pass
            scores = model(getLongTensor(np.array([seqs]))[0], getFloatTensor(image_features), getLongTensor(np.array(lengths)))
            targets = getLongTensor(np.array([idxs]))
            loss = nn.CrossEntropyLoss()
            output = loss(scores[:, :, 0], targets[0])
            train_loss += output.data[0]
            
            # backward pass
            model.zero_grad()
            output.backward()
            
            # update weights
            optimizer.step()
                        
            if(updates % 1000 == 0):
                print("update: {}, train_loss: {}, time {}".format(updates, train_loss/updates, time.time()-start))
        
        train_loss_list.append(train_loss/updates)
        print("iter %r: avg train loss=%.4f, time=%.2fs" % (ITER, train_loss/updates, time.time()-start))
        top1, top5 = evaluate(model, val, show_n_wrong_images=0, show_n_good_images=0)   
        top1_val_list.append(top1)
        top5_val_list.append(top5)
        
        print("VALIDATION: TOP 1: {}, TOP 5: {}".format(top1, top5))
        top1, top5 = evaluate(model, train[:1000], show_n_wrong_images=0, show_n_good_images=0)  
        top1_train_list.append(top1)
        top5_train_list.append(top5)

except KeyboardInterrupt:
    print('Stopped at ITER: ' + str(ITER))

# create_learning_curves(top1_val_list, top5_val_list, top1_train_list, top5_train_list)
top1, top5 = evaluate(model, val, show_n_wrong_images=0, show_n_good_images=0)
print("TOP 1: {}, TOP 5: {}".format(top1, top5))


write(top1_val_list, "top1_val")
write(top5_val_list, "top5_val")
write(top1_train_list, "top1_train")
write(top5_train_list, "top5_train")
write(train_loss_list, "train_loss")

top1, top5 = evaluate(model, test, show_n_wrong_images=0, show_n_good_images=0)
print("TOP 1: {}, TOP 5: {}".format(top1, top5))
write(top1, "top1_test")
write(top5, "top5_test")


iter 0: avg train loss=2.2186, time=4.43s
VALIDATION: TOP 1: 0.1494, TOP 5: 0.6664
iter 1: avg train loss=2.0396, time=4.43s
VALIDATION: TOP 1: 0.185, TOP 5: 0.7256
iter 2: avg train loss=1.9483, time=4.42s
VALIDATION: TOP 1: 0.2142, TOP 5: 0.7706
iter 3: avg train loss=1.8700, time=4.55s
VALIDATION: TOP 1: 0.2318, TOP 5: 0.7924
iter 4: avg train loss=1.7789, time=4.37s
VALIDATION: TOP 1: 0.2562, TOP 5: 0.7904
iter 5: avg train loss=1.6757, time=4.81s
VALIDATION: TOP 1: 0.264, TOP 5: 0.7942
iter 6: avg train loss=1.5732, time=4.87s
VALIDATION: TOP 1: 0.2692, TOP 5: 0.8086
iter 7: avg train loss=1.4656, time=4.50s
VALIDATION: TOP 1: 0.2854, TOP 5: 0.8174
iter 8: avg train loss=1.3600, time=4.58s
VALIDATION: TOP 1: 0.276, TOP 5: 0.8162
iter 9: avg train loss=1.2628, time=4.63s
VALIDATION: TOP 1: 0.2922, TOP 5: 0.8238
iter 10: avg train loss=1.1573, time=4.50s
VALIDATION: TOP 1: 0.281, TOP 5: 0.8026
iter 11: avg train loss=1.0661, time=4.49s
VALIDATION: TOP 1: 0.296, TOP 5: 0.8284
iter 12

VALIDATION: TOP 1: 0.3086, TOP 5: 0.8342
iter 99: avg train loss=0.0667, time=4.45s
VALIDATION: TOP 1: 0.3036, TOP 5: 0.8344
TOP 1: 0.304, TOP 5: 0.8346
TOP 1: 0.3108, TOP 5: 0.8366


In [137]:
create_learning_curves(top1_val_list, top5_val_list, top1_train_list, top5_train_list)

In [127]:
# plt.plot(x, y)
# plt.show()


# torch.cuda.clear() 
torch.cuda.empty_cache() 

In [56]:
# rnn = torch.nn.LSTM(input_size=300, hidden_size=100, batch_first=True)
# outputs, h_n = rnn(Variable(torch.rand(75, 100, 300))) # input is of shape [batch_size, sequence, features]
# print(outputs)

# question_lengths = Variable(torch.LongTensor([2,3,4]))
# last_out = torch.gather(outputs, 1, question_lengths.view(-1,1,1).expand(3,1,100)-1) # minus 1 to get from length to index
# print(last_out)

Variable containing:
( 0 ,.,.) = 
 -6.2118e-02  7.9667e-02 -5.2082e-02  ...   9.5811e-02 -7.9205e-02  1.0630e-01
 -5.5698e-02  1.7827e-02 -1.3496e-01  ...  -6.1180e-02 -1.2932e-01  1.3219e-01
 -8.9346e-02 -4.1923e-02 -1.3960e-02  ...  -7.9432e-02 -1.0256e-01  2.4854e-01
                 ...                   ⋱                   ...                
 -9.3857e-02 -2.1711e-01 -1.1161e-01  ...   3.1571e-02 -6.1161e-02  3.1064e-01
 -8.4089e-02 -1.2788e-01 -2.1182e-01  ...  -1.1321e-01 -1.3140e-01  3.4619e-01
 -5.5830e-02  1.1704e-01  3.4745e-02  ...  -1.6995e-01 -1.2687e-01  2.7499e-01

( 1 ,.,.) = 
 -1.1401e-02  8.0521e-02  9.2011e-02  ...   4.3114e-02 -1.7811e-02  9.4482e-02
 -7.0157e-02  3.7095e-02  1.0413e-01  ...  -7.8126e-02 -6.2758e-02  2.9297e-01
 -7.2522e-02  3.9287e-02  1.4407e-01  ...  -1.3843e-01 -4.9256e-02  2.7699e-01
                 ...                   ⋱                   ...                
 -1.2916e-01 -1.3953e-01  3.3196e-02  ...  -1.2510e-01 -1.3220e-01  2.8005e-01
 -1.